In [213]:
import random

import numpy
from tabulate import tabulate

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

import warnings

warnings.filterwarnings("ignore")

In [214]:
random.seed(2022)

In [215]:
N_QUEENS = 15

In [216]:
def evalQueens(individual):
    """Evaluation function for the n-queens problem.
    The problem is to determine a configuration of n queens
    on a nxn chessboard such that no queen can be taken by
    one another. Each queens are assigned
    to one column, and only one queen can be on each line.
    The evaluation function therefore only counts the number
    of conflicts along the diagonals.
    """

    #Count the number of conflicts with other queens.

    #The conflicts can only be diagonal, count on each diagonal line
    left_diagonal = [0] * (2*N_QUEENS-1)
    right_diagonal = [0] * (2*N_QUEENS-1)

    #Sum the number of queens on each diagonal:
    for i in range(N_QUEENS):
        left_diagonal[i+individual[i]] += 1
        right_diagonal[N_QUEENS-1-i+individual[i]] += 1

    #Count the number of conflicts on each diagonal
    sum_ = 0
    for i in range(2*N_QUEENS-1):
        if left_diagonal[i] > 1:
            sum_ += left_diagonal[i] - 1
        if right_diagonal[i] > 1:
            sum_ += right_diagonal[i] - 1
    return sum_,

In [217]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

#Since there is only one queen per line,
#individual are represented by a permutation
toolbox = base.Toolbox()
toolbox.register("permutation", random.sample, range(N_QUEENS), N_QUEENS)

#Structure initializers
#An individual is a list that represents the position of each queen.
#Only the line is stored, the column is the index of the number in the list.
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.permutation)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evalQueens)
toolbox.register("mate", tools.cxPartialyMatched)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=1.0/N_QUEENS)
toolbox.register("select", tools.selTournament, tournsize=4)

In [218]:
pop = toolbox.population(n=300)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("Avg", numpy.mean)
stats.register("Std", numpy.std)
stats.register("Min", numpy.min)
stats.register("Max", numpy.max)

log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=1000, stats=stats,
                    halloffame=hof, verbose=True)

gen	nevals	Avg 	Std    	Min	Max
0  	300   	7.68	1.89322	2  	14 
1  	181   	6.58	1.75031	1  	12 
2  	192   	6.16333	1.8735 	1  	12 
3  	187   	5.51667	1.82658	1  	11 
4  	188   	5.25   	2.06418	1  	11 
5  	181   	4.45333	2.08834	1  	11 
6  	165   	3.70667	2.17423	1  	11 
7  	199   	3.49667	2.34023	1  	11 
8  	168   	2.64   	2.11906	1  	9  
9  	188   	1.74667	1.68794	1  	11 
10 	177   	1.41   	1.13221	1  	9  
11 	173   	1.45333	1.20878	1  	7  
12 	186   	1.42   	1.21529	1  	9  
13 	170   	1.35   	1.10491	1  	8  
14 	181   	1.43333	1.27497	1  	8  
15 	177   	1.34   	1.0317 	1  	7  
16 	174   	1.37667	1.18945	1  	7  
17 	184   	1.34333	0.972345	1  	5  
18 	196   	1.42667	1.12159 	1  	8  
19 	189   	1.34667	1.03914 	1  	7  
20 	172   	1.35   	1.15217 	1  	8  
21 	179   	1.45667	1.28379 	1  	7  
22 	191   	1.40667	1.21434 	1  	10 
23 	164   	1.40667	1.25484 	1  	9  
24 	183   	1.40333	1.12279 	1  	8  
25 	191   	1.29667	1.03376 	1  	9  
26 	188   	1.41333	1.09962 	1  	7  
27 	194   	1.39667	

In [219]:
best = hof[0]
best

[6, 8, 11, 1, 12, 7, 5, 0, 2, 14, 10, 3, 13, 9, 4]

In [220]:
def display_positional_grid(individual):
    # construct board using pandas
    board = [[""]*(N_QUEENS + 1) for _ in range(N_QUEENS)]

    for i in range(N_QUEENS):
        board[i][0] = i + 1

    # draw all conflicts with a red line
    for x in range(N_QUEENS):
        x_row, x_column = individual[x], x
        for y in range(x + 1, N_QUEENS):
            y_row, y_column = individual[y], y

            diff_row, diff_column = y_row - x_row, y_column - x_column
            # check if queens are conflicting
            if x_row == y_row or x_column == y_column or abs(diff_row) == abs(diff_column):
                # draw a line of the conflict
                for i in range(1 + max(abs(diff_row), abs(diff_column))):
                    board[x_column + i * numpy.sign(diff_column)][1 + x_row + i * numpy.sign(diff_row)] = "🟥"

    # draw all queens
    for column, row in enumerate(individual):
        board[column][1 + row] = "👑" if board[column][1 + row] == "" else "♕"

    header = list(range(1,N_QUEENS + 1))
    print(tabulate(board, headers=header, tablefmt="grid"))

In [221]:
best

[6, 8, 11, 1, 12, 7, 5, 0, 2, 14, 10, 3, 13, 9, 4]

In [222]:
display_positional_grid(best)

+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+
|    | 1   | 2   | 3   | 4   | 5   | 6   | 7   | 8   | 9   | 10   | 11   | 12   | 13   | 14   | 15   |
+====+=====+=====+=====+=====+=====+=====+=====+=====+=====+======+======+======+======+======+======+
|  1 |     |     |     |     |     |     | 👑  |     |     |      |      |      |      |      |      |
+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+
|  2 |     |     |     |     |     |     |     |     | 👑  |      |      |      |      |      |      |
+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+
|  3 |     |     |     |     |     |     |     |     |     |      |      | 👑   |      |      |      |
+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+
|  4 |     | 👑  |     |     |     |     |     |     |     |      |      |   

In [223]:
bad = toolbox.individual()
bad

[14, 4, 12, 0, 8, 5, 6, 11, 10, 3, 7, 9, 2, 13, 1]

In [224]:
display_positional_grid(bad)

+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+
|    | 1   | 2   | 3   | 4   | 5   | 6   | 7   | 8   | 9   | 10   | 11   | 12   | 13   | 14   | 15   |
+====+=====+=====+=====+=====+=====+=====+=====+=====+=====+======+======+======+======+======+======+
|  1 |     |     |     |     |     |     |     |     |     |      |      |      |      |      | ♕    |
+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+
|  2 |     |     |     |     | 👑  |     |     |     |     |      |      |      |      | 🟥   |      |
+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+
|  3 |     |     |     |     |     |     |     |     |     |      |      |      | ♕    |      |      |
+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+
|  4 | ♕   |     |     |     |     |     |     |     |     |      |      | 